In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, CategoricalParameter, ContinuousParameter
import s3fs

import os
import json

In [ ]:
sagemaker_session = sagemaker.Session()

bucket = 'animal-reidentification'

role = sagemaker.get_execution_role()

In [ ]:
print(bucket, role)
print(os.getcwd())

In [ ]:
data_location = f's3://{bucket}/data/'
annotations_location = f's3://{bucket}/annotations/'

print(f'data_location: {data_location}')
print(f'annotations_location: {annotations_location}')

## For running locally

import os
import subprocess

local_instance_type = "local"

try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        local_instance_type = "local_gpu"
except:
    pass

print("local_instance_type = " + local_instance_type)

## Kick off the instance

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='denseNet201_secondPass.py',
                    source_dir='./kevin/',
                    framework_version='1.8.0',
                    role=role,
                    py_version='py3',
                    instance_count=1,
                    instance_type='ml.g4dn.xlarge', # Sagemaker managed instance
#                     instance_type=local_instance_type, # local notebook instance
                    hyperparameters={
                        'batch-size': 32,
                        'epochs': 1,
                        'name': 'trialRun_10000tr_1500val_10epoch_batch16',
#                         'save-model': True,
                    })

estimator.fit({
    'data': data_location,
    'annotations': annotations_location
})